In [ ]:
"""
Code to prep data for t5 modeling. 
I/p to t5: <prefix>:<sentence>
I/p to bert: <sentence> [SEP] <hugchat explanation> [SEP] <keywords> (not sure about keywords) -> Predict hate_speech
"""

In [14]:
import pandas as pd
import numpy as np
import ast

In [15]:
### Input 
train_processed_path = '../data/rationale_extraction/df_train_rationale_post_processed.csv'
val_processed_path = '../data/rationale_extraction/df_val_rationale_post_processed.csv'
test_processed_path = '../data/rationale_extraction/df_test_rationale_post_processed.csv'

### Output 
train_t5_path = '../data/t5_modeling/df_train.csv'
val_t5_path = '../data/t5_modeling/df_val.csv'
test_t5_path = '../data/t5_modeling/df_test.csv'

train_bert_path = '../data/bert_modeling/df_train.csv'
val_bert_path = '../data/bert_modeling/df_val.csv'
test_bert_path = '../data/bert_modeling/df_test.csv'

In [16]:
def load_data(path):
    df = pd.read_csv(path)
    label_mapper = {'hate_speech':'hate', 'normal_speech':'normal', 'offensive_speech':'offensive'}
    df['hugchat_label_processed'] = df['hugchat_label_processed'].map(label_mapper)
    return df

In [17]:
def get_label_helper(annotator_response):
    labels = ast.literal_eval(annotator_response)['label']
    count_hate = labels.count(0)
    count_offensive = labels.count(2)
    count_normal = labels.count(1)
    if count_hate==count_offensive==count_normal:
        return "discard"
    elif count_hate > max(count_offensive, count_normal):
        return "hate"
    elif count_offensive > max(count_hate, count_normal):
        return "offensive"
    elif count_normal > max(count_hate, count_offensive):
        return "normal"
    else:
        return "discard"

# 0 = Hate, 1 = normal, 2 = offensive 
def get_label(df):
    df['gt_label'] = df['annotators'].apply(lambda x: get_label_helper(x))

In [18]:
def get_comm_target_helper(annotator_response):
    comm_target = ast.literal_eval(annotator_response)['target']
    comm_target_all = set()
    for tgt in comm_target:
        for comm in tgt: 
            if comm not in comm_target_all:
                comm_target_all.add(comm)
    return list(comm_target_all)
    
# 0 = Hate, 1 = normal, 2 = offensive 
def get_comm_target(df):
    df['gt_comm_target'] = df['annotators'].apply(lambda x: get_comm_target_helper(x))

In [19]:
def get_rationales_helper(rationales, tokens):
    rationales = ast.literal_eval(rationales)
    if len(rationales)==0:
        return []
    ratnl_sum = np.array(rationales).sum(axis=0).tolist()
    ind = [i for i in range(len(ratnl_sum)) if ratnl_sum[i]>0]
    tokens = ast.literal_eval(tokens)
    rational_tokens = [tokens[i] for i in ind]
    return rational_tokens
    
def get_rationales(df):
    df['gt_keywords'] = df[['rationales','post_tokens']].apply(lambda row: get_rationales_helper(row['rationales'], row['post_tokens']), axis=1)
    

In [20]:
# Function to create column with mixed community target 
# if there are no keywords in gt_rationales, we pick keywords from hugchat_keywords_processed
def create_mix_keywords(df):
    df['mix_keywords'] = df['gt_keywords'].copy()
    no_kw_idx = df['mix_keywords'].apply(lambda x: len(x)==0)
    df['mix_keywords'][no_kw_idx] = df['hugchat_keywords_processed'][no_kw_idx]

In [21]:
# T5 data should consist of <prefix> <sentence>
# <label> <sentence> - classification task to predict hate speech label
# <explanation> <sentence> - generate free-text explanation 
# <keywords> <sentence> - generate keywords 
# <comm target> <sentence> - comm. target  
def create_t5_data(df):
    # hate speech classification 
    df_classification = df[['id','unmasked_sentence','gt_label']].copy()
    df_classification['prefix'] = 'label'
    df_classification = df_classification[['id','prefix','unmasked_sentence','gt_label']].copy()
    df_classification.rename(columns = {'unmasked_sentence':'input_text', 'gt_label':'target_text'}, inplace=True)
    print("Classification shape: ", df_classification.shape)
    
    # generate explanation 
    df_explanation = df[['id','unmasked_sentence','hugchat_explanation']].copy()
    df_explanation['prefix'] = 'explanation'
    drop_row_idx = df['hugchat_explanation'][df['hugchat_explanation'].apply(str).apply(lambda x: len(x)<=10)].index
    df_explanation.drop(index=drop_row_idx, inplace=True)
    df_explanation.reset_index(drop=True, inplace=True)
    df_explanation = df_explanation[['id','prefix','unmasked_sentence','hugchat_explanation']].copy()
    df_explanation.rename(columns = {'unmasked_sentence':'input_text', 'hugchat_explanation':'target_text'}, inplace=True)
    print("Explanation shape: ", df_explanation.shape)
    
    # get communities targeted
    df_comm_target = df[['id','unmasked_sentence','gt_comm_target']].copy()
    df_comm_target['prefix'] = 'comm_target'
    df_comm_target['gt_comm_target'] = df_comm_target['gt_comm_target'].apply(lambda x: ' '.join(k for k in x))
    df_comm_target = df_comm_target[['id','prefix','unmasked_sentence','gt_comm_target']].copy()
    df_comm_target.rename(columns = {'unmasked_sentence':'input_text', 'gt_comm_target':'target_text'}, inplace=True)
    print("Comm Target shape: ", df_comm_target.shape)
    
    # get keywords 
    df_kw = df[['id','unmasked_sentence', 'mix_keywords']].copy()
    df_kw['prefix'] = 'keywords'
    df_kw['mix_keywords'] = df_kw['mix_keywords'].apply(str).apply(lambda x: ' '.join(k for k in ast.literal_eval(x)))
    df_kw = df_kw[['id','prefix','unmasked_sentence','mix_keywords']].copy()
    df_kw.rename(columns = {'unmasked_sentence':'input_text', 'mix_keywords':'target_text'}, inplace=True)
    print("Keywords shape: ", df_comm_target.shape)
    
    df_t5 = pd.concat([df_classification, df_explanation, df_comm_target, df_kw], ignore_index=True)
    print("Final shape: ", df_t5.shape)
    df_t5 = df_t5.sample(frac=1, random_state=42)
    df_t5.reset_index(drop=True, inplace=True)
    print("\n")
    return df_t5

In [22]:
## create bert data 
## objective: predict hate speech and communities targeted from sentence, hugchat rationale and keywords 
def create_bert_data(df):
    df_bert = df[['id','unmasked_sentence','hugchat_keywords_processed','hugchat_explanation','mix_keywords','gt_comm_target','gt_label']].copy()
    drop_row_idx = df['hugchat_explanation'][df['hugchat_explanation'].apply(str).apply(lambda x: len(x)<=10)].index
    df_bert.drop(index=drop_row_idx, inplace=True)
    df_bert.reset_index(drop=True, inplace=True)
    print("Bert data shape: ", df_bert.shape)
    return df_bert 

In [23]:
def run(path):
    df = load_data(path)
    get_label(df)
    get_comm_target(df)
    get_rationales(df)
    create_mix_keywords(df)
    df_t5 = create_t5_data(df)
    df_bert = create_bert_data(df)
    return df_t5, df_bert  

In [24]:
df_t5_train, df_bert_train = run(train_processed_path)
df_t5_val, df_bert_val = run(val_processed_path)
df_t5_test, df_bert_test = run(test_processed_path)

Classification shape:  (14072, 4)
Explanation shape:  (14057, 4)
Comm Target shape:  (14072, 4)
Keywords shape:  (14072, 4)
Final shape:  (56273, 4)


Bert data shape:  (14057, 7)
Classification shape:  (1787, 4)
Explanation shape:  (1786, 4)
Comm Target shape:  (1787, 4)
Keywords shape:  (1787, 4)
Final shape:  (7147, 4)


Bert data shape:  (1786, 7)
Classification shape:  (1761, 4)
Explanation shape:  (1759, 4)
Comm Target shape:  (1761, 4)
Keywords shape:  (1761, 4)
Final shape:  (7042, 4)


Bert data shape:  (1759, 7)


In [25]:
## Save the outputs 
df_t5_train.to_csv(train_t5_path, index=False)
df_t5_val.to_csv(val_t5_path, index=False)
df_t5_test.to_csv(test_t5_path, index=False)

df_bert_train.to_csv(train_bert_path, index=False)
df_bert_val.to_csv(val_bert_path, index=False)
df_bert_test.to_csv(test_bert_path, index=False)